# Carregamento da base de dados

In [ ]:
# Carregamento da base de dados
import pandas as pd
import matplotlib.pyplot as plt # Geração de gráficos
import seaborn as sns # Geração de gráficos
import numpy as np

# Carregamento
casas_file = pd.read_csv('./csv-files/house_prices.csv')

# Selecionando apenas colunas com valores numéricos
base_casas = casas_file.select_dtypes(include=[float, int])

## Valores Nulos

In [ ]:
#False - Nao possui valores nulors / True - Possue algum valor nulo
base_casas.isnull()
base_casas.isnull().sum() # Numero de valores nulos de cada coluna

# Visualização com Gráficos

In [ ]:
figura = plt.figure(figsize=(20,20))
sns.heatmap(base_casas.corr(), annot= True)

# Pré Processamento

In [ ]:
base_casas.columns

In [ ]:
# da coluna Bedrooms até Long - .values para transformar no formato do numpy
X = base_casas.iloc[:, 2:18].values
X[0]

In [ ]:
Y = base_casas.iloc[:, 1]

Dados estao no formato de vetor, mas é necessario mandar para AWS em formato de matriz. Necessarior realizar a conversao 

In [ ]:
X = np.array(X).astype('float32')
Y = np.array(Y).astype('float32')

Divisao entre: Porção de treinamento x Porção de testagem 

In [ ]:
from sklearn.model_selection import train_test_split
# Teste = 30% - Treinamento = 70%
X_treinamento, X_teste, Y_treinamento, Y_teste = train_test_split(X, Y, test_size=0.3, random_state=0)

In [ ]:
X_treinamento.shape, X_teste.shape, Y_treinamento.shape, Y_teste.shape

# Configurações SageMaker

In [ ]:
import sagemaker
import boto3
from sagemaker import Session

session = sagemaker.Session() # Iniciando session

bucket = 'sagemaker-curso-bucket' # Bucket S3
subpasta_modelo = 'modelos/house-prices/linear-learner'
subpasta_dataset = 'datasets/house-prices'


In [ ]:
# Ao criar o ambiente no sagemaker, criamos essas regras
role = sagemaker.get_execution_role()
print(role)

In [ ]:
import io
import sagemaker.amazon.common as smac # sagemaker commom library



buffer = io.BytesIO()
smac.write_numpy_to_dense_tensor(buffer, X_treinamento, Y_treinamento) # convertendo numpy to aws format
buffer.seek(0)

Upload para o Bucket S3

In [ ]:
import os
key = 'houses-train-data'

## os path faz a concatenação do caminho
boto3.resource('s3').Bucket(bucket).Object(os.path.join(subpasta_dataset, 'train', key)).upload_fileobj(buffer)

In [ ]:
s3_train_data = 's3://{}/{}/train/{}'.format(bucket, subpasta_dataset, key)
print('Localização da base de treinamento: {}'.format(s3_train_data))

In [ ]:
# onde salvar o modelo depois de treinado
output_location = 's3://{}/{}/output'.format(bucket, subpasta_modelo)
print('Modelo será salvo em: {}'.format(output_location))

# Criação e treinamento do Linear Learner

In [ ]:
import sagemaker.image_uris


container = sagemaker.image_uris.retrieve(framework='linear-learner', region=boto3.Session().region_name)


In [ ]:
linear = sagemaker.estimator.Estimator(image_uri=container,
                                       role = "AmazonSageMaker-ExecutionRole-20240702T173175",
                                       instance_count= 1,
                                       instance_type='ml.m4.xlarge',
                                       output_path= output_location,
                                       sagemaker_session= session
                                    )

In [ ]:
# Feature_dim = Numero de parametros
# Num_models = Numeros de modelos a serem treinados. O melhor será escolhido ao final
linear.set_hyperparameters(feature_dim = 16,
                           predictor_type = 'regressor',
                           num_models = 16
                           )

In [ ]:
# Localização da base de dados de treinamento
linear.fit({'train': s3_train_data})

# Deploy, Previsões e avaliação

In [ ]:
linear_regresssor = linear.deploy(initial_instance_count=1,
                                  instance_type='ml.m4.xlarge'
                                  )

-> Recomendação de não utilizar SageMaker Endpoints devido ao custo.

## Previsões (apenas código, sem deploy real no sageMaker)

In [ ]:
from sagemaker.serializers import CSVSerializer # Formato de dados que o modelo recebe como parametro 
from sagemaker.deserializers import JSONDeserializer

linear_regresssor.serializer = CSVSerializer()
linear_regresssor.deserializer = JSONDeserializer()

In [ ]:
results = linear_regresssor.predict(X_teste)
results

In [ ]:
previsoes = np.array([r ['score'] for r in results ['predictions']])
previsoes
previsoes.shape, X_teste.shape

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
mae = mean_absolute_error(Y_teste, previsoes)
mse = mean_squared_error(Y_teste, previsoes)
print("MAE =", mae, "\MSE = ", mse)